<a href="https://colab.research.google.com/github/donghui-0126/mini-project/blob/main/shoes-project/resell_regressor/bn_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LinearRegression
import warnings


warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
df = pd.read_csv('/content/drive/MyDrive/kream_data/product_data_dpp.csv')
df.head(3)

,product_id,img_path,brand,name,color1,color2,price_og,price_resell,n_scrap,+price,tokenized_color,tokenized_bn
0,28029,crawling\product_crawling\image\28029.jpg,Nike,Nike Dunk Low Retro Black,WHITE,BLACK,129000,152800,123000.0,23800,[0 0 0 0 0 0 0 0 0 0 2 4 3 5],[ 0 0 0 0 0 0 0 0 0 0 0 0 0 2 4 ...
1,12831,crawling\product_crawling\image\12831.jpg,Nike,Nike Air Force 1 '07 Low White,WHITE,WHITE,139000,136200,151000.0,-2800,[0 0 0 0 0 0 0 0 0 0 2 4 3 4],[ 0 0 0 0 0 0 0 0 0 0 0 2 4 3 4 ...
2,21935,crawling\product_crawling\image\21935.jpg,Nike,Nike Air Force 1 '07 WB Flax,FLAX,GUM,169000,173200,69000.0,4200,[ 0 0 0 0 0 0 0 0 0 0 2 82 3 51],[ 0 0 0 0 0 0 0 0 0 0 0 ...


In [16]:
df.shape

(6103, 12)

In [17]:
bn_input_data = np.array([])

for val in df['tokenized_bn'].values:
    val = [int(x) for x in val.strip('[]').split()]
    bn_input_data = np.append(bn_input_data, val)

bn_input_data = bn_input_data.reshape([6103, 21])
bn_input_data = bn_input_data.astype(int)

In [26]:
bn_input_data.shape

(6103, 21)

In [38]:
columns = [x for x in range(100)]
index = [y for y in range(bn_input_data.shape[0])]


bn_input_df = pd.DataFrame(index=index, columns=columns)
bn_input_df = bn_input_df.fillna(0)

for index, line in enumerate(bn_input_data):
    for elem in set(line):
        if elem < 100:
            bn_input_df.iloc[index,elem] = 1
        else:
            bn_input_df.iloc[index,0] = 1

In [54]:
# 신발의 브랜드와 이름은 리셀가격(신발의 본래 가치??)와 관련이 클 것 같아서 target데이터를 price_resell 로 설정함. 

target_data = np.array(df['price_resell'])
print(target_data.shape)

(6103,)


In [57]:
target_data.astype(int)

array([152800, 136200, 173200, ..., 450000, 457500, 400000])

In [132]:
# PCA의 차원을 늘리면서  K-fold를 사용해서 어떤 모델이 가장 효율적인지 살펴보겠다. 
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

for n in [20]:
    pca = PCA(n_components=n)
    bn_pca = pca.fit_transform(bn_input_df)
    bn_pca_df = pd.DataFrame(bn_pca)

    kf = KFold(n_splits = 5)
    
    rmse_list = []
    for train_index, test_index in kf.split(target_data):
        bn_pca_df_train, bn_pca_df_test = bn_pca_df.loc[train_index], bn_pca_df.loc[test_index]
        target_data_train, target_data_test  = target_data[train_index], target_data[test_index]

        reg = LinearRegression().fit(bn_pca_df_train, target_data_train)

        pred = reg.predict(bn_pca_df_test)
        rmse = mean_squared_error(pred, target_data_test, squared=False)
        rmse_list.append(rmse)
        # print(rmse)
        
    print("PCA:", n, "rmse:", round(np.mean(rmse_list)))

PCA: 20 rmse: 285222


In [108]:
# PCA의 차원을 늘리면서  K-fold를 사용해서 어떤 모델이 가장 효율적인지 살펴보겠다. 
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

for n in (80, 90, 100):
    pca = PCA(n_components=n)
    bn_pca = pca.fit_transform(bn_input_df)
    bn_pca_df = pd.DataFrame(bn_pca)

    kf = KFold(n_splits = 5)
    
    rmse_list = []
    for train_index, test_index in kf.split(target_data):
        bn_pca_df_train, bn_pca_df_test = bn_pca_df.loc[train_index], bn_pca_df.loc[test_index]
        target_data_train, target_data_test  = target_data[train_index], target_data[test_index]
        
        lgbm_reg = LGBMRegressor().fit(bn_pca_df_train, target_data_train)

        pred = lgbm_reg.predict(bn_pca_df_test)
        rmse = mean_squared_error(pred, target_data_test, squared=False)
        rmse_list.append(rmse)
        # print(rmse)
        
    print("PCA:", n, "rmse:", round(np.mean(rmse_list)))

PCA: 80 rmse: 284168
PCA: 90 rmse: 283963
PCA: 100 rmse: 279354


In [104]:
kf = KFold(n_splits = 5)

rmse_list = []
for train_index, test_index in kf.split(target_data):
    bn_pca_df_train, bn_pca_df_test = bn_input_df.loc[train_index], bn_input_df.loc[test_index]
    target_data_train, target_data_test  = target_data[train_index], target_data[test_index]

    model = keras.models.Sequential([
            keras.layers.Dense(32),
            keras.layers.Dense(8),
            keras.layers.Dense(1)
    ])

    model.compile(loss="mse")
    model.fit(bn_pca_df_train,target_data_train, epochs=20, verbose=0)
    pred = model.predict(bn_pca_df_test, verbose=0)
    rmse = mean_squared_error(target_data_test,pred, squared= False)

    rmse_list.append(rmse)
print("PCA:", n, "rmse:", round(np.mean(rmse_list)))

PCA: 100 rmse: 345963


여러 가지를 실험해봤는데....

- 주성분을 20으로 축소한 선형회귀 모델 
- 주성분을 아주 조금 축소하거나 원본데이터 그대로인 lgbm 모델이 적절할듯 하다.

In [134]:
# 차라리 이모델이 좋긴한데... 마지막 3개가 너무 가격이 비정상적이다.
# 이거 word_num 처리할때 날라간것 같다. 다시 수정해보자.

reg.predict(bn_pca)

array([296435.81089691, 262560.70950202, 218667.21338149, ...,
       643871.98779304, 714860.89851604, 908089.25514142])

In [136]:
lgbm_reg.predict(bn_input_df)

array([373269.39229005, 440063.13102925, 440063.13102925, ...,
       154663.72864104, 203405.41830329, 156426.99033297])